<a href="https://colab.research.google.com/github/mugalan/energy-plus-utility/blob/dev/BMS_Cookbook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install

In [ ]:
# install from your dev branch
!pip install -q "energy-plus-utility @ git+https://github.com/mugalan/energy-plus-utility.git@dev"

In [ ]:
# run the silent bootstrap in this kernel
from eplus import prepare_colab_eplus
prepare_colab_eplus()  # raises on failure, otherwise silent

# now your imports work
from eplus import EPlusUtil, EPlusSqlExplorer

## Initialize class

In [ ]:
import subprocess, json, pathlib, os
import pandas as pd
EPLUS = str(pathlib.Path.home() / "EnergyPlus-25-1-0")
EPLUS_ROOT = "/root/EnergyPlus-25-1-0"

out_dir = "/content/eplus_out"
idf = f"{EPLUS}/ExampleFiles/5ZoneAirCooled.idf"
epw = f"{EPLUS}/WeatherData/USA_CA_San.Francisco.Intl.AP.724940_TMY3.epw"

util = EPlusUtil(verbose=1, out_dir = out_dir)
util.delete_out_dir()
util.set_model(idf,epw)
state = util.api.state_manager.new_state()
util.ensure_output_sqlite()

## *Optional:* Convert the model to .json

In [ ]:
idf_path = pathlib.Path(idf)
converter = os.path.join(EPLUS_ROOT, "ConvertInputFormat")   # on Windows it's ConvertInputFormat.exe

# Convert IDF → epJSON (outputs 5ZoneAirCooled.epJSON in the same folder)
subprocess.run([converter, str(idf_path)], check=True)

epjson_path = idf_path.with_suffix(".epJSON")
print("epJSON exists?", epjson_path.exists(), epjson_path)

## Simulations

###Dry run to create tables etc.

In [ ]:
util = EPlusUtil(verbose=1)
util.delete_out_dir()
util.set_model(idf,epw)
util.ensure_output_sqlite()
util.dry_run_min(include_ems_edd=False)

In [ ]:
util.list_zone_names()

In [ ]:
pd.DataFrame(util.list_variables_safely())

In [ ]:
pd.DataFrame(util.list_actuators_safely())

## Setup Simulation

### Enable Varaibles and Meters

In [ ]:
util.ensure_output_variables([
    # {"name":"Site Outdoor Air Drybulb Temperature", "key":"", "freq":"TimeStep"},
    # {"name":"Site Outdoor Air Dewpoint Temperature", "key":"", "freq":"TimeStep"},
    # {"name":"Site Wind Speed", "key":"", "freq":"TimeStep"},
    # {"name":"Site Diffuse Solar Radiation Rate per Area", "key":"", "freq":"TimeStep"},
    # {"name":"Site Direct Solar Radiation Rate per Area", "key":"", "freq":"TimeStep"},
    {'name':'Zone Mean Air Temperature',          'key':'*',           'freq':'TimeStep'},
    {'name':'Zone Mean Air Dewpoint Temperature', 'key':'*',           'freq':'TimeStep'},
    {'name':'Zone Air Relative Humidity',          'key':'*',           'freq':'TimeStep'},
    {'name':'Zone People Occupant Count',         'key':'*',           'freq':'TimeStep'},
    {'name':'Site Outdoor Air Drybulb Temperature','key':'Environment','freq':'TimeStep'},
    {'name':'Site Outdoor Air Wetbulb Temperature','key':'Environment','freq':'TimeStep'},
    {"name":"Zone Air System Sensible Cooling Energy", "key":"*", "freq":"TimeStep"},
    {"name":"Zone Total Internal Latent Gain Energy", "key":"*", "freq":"TimeStep"},
    {"name": "Zone Air CO2 Concentration", "key": "*", "freq": "TimeStep"}
    # {"name":"Zone Air System Sensible Cooling Energy", "key":"SPACE2-1", "freq":"TimeStep"},
    # {"name":"Zone Air System Sensible Cooling Energy", "key":"SPACE3-1", "freq":"TimeStep"},
    # {"name":"Zone Air System Sensible Cooling Energy", "key":"SPACE4-1", "freq":"TimeStep"},
    # {"name":"Zone Air System Sensible Cooling Energy", "key":"SPACE5-1", "freq":"TimeStep"},
], activate=True)


In [ ]:
# 1) Ensure the meter(s) you want are reported
output_meteres = ["InteriorLights:Electricity:Zone:SPACE5-1","Cooling:EnergyTransfer:Zone:SPACE1-1","Cooling:EnergyTransfer","Electricity:Facility","ElectricityPurchased:Facility", "ElectricitySurplusSold:Facility"]

util.ensure_output_meters(output_meteres, freq="TimeStep")


### Run Simulation

In [ ]:
util = EPlusUtil(verbose=1)
util.delete_out_dir()
util.set_model(idf,epw, outdoor_co2_ppm=0.0, per_person_m3ps_per_W=0.0) #3.82e-08)
# util.prepare_annual_run_with_co2()
# util.ensure_output_variables([
#     # {"name":"Site Outdoor Air Drybulb Temperature", "key":"", "freq":"TimeStep"},
#     # {"name":"Site Outdoor Air Dewpoint Temperature", "key":"", "freq":"TimeStep"},
#     # {"name":"Site Wind Speed", "key":"", "freq":"TimeStep"},
#     # {"name":"Site Diffuse Solar Radiation Rate per Area", "key":"", "freq":"TimeStep"},
#     # {"name":"Site Direct Solar Radiation Rate per Area", "key":"", "freq":"TimeStep"},
#     {'name':'Zone Mean Air Temperature',          'key':'*',           'freq':'TimeStep'},
#     {'name':'Zone Mean Air Dewpoint Temperature', 'key':'*',           'freq':'TimeStep'},
#     {'name':'Zone Air Relative Humidity',          'key':'*',           'freq':'TimeStep'},
#     {'name':'Zone People Occupant Count',         'key':'*',           'freq':'TimeStep'},
#     {'name':'Site Outdoor Air Drybulb Temperature','key':'Environment','freq':'TimeStep'},
#     {'name':'Site Outdoor Air Wetbulb Temperature','key':'Environment','freq':'TimeStep'},
#     {"name":"Zone Air System Sensible Cooling Energy", "key":"*", "freq":"TimeStep"},
#     {"name":"Zone Total Internal Latent Gain Energy", "key":"*", "freq":"TimeStep"},
#     {"name": "Zone Air CO2 Concentration", "key": "*", "freq": "TimeStep"},
#     {"name": "Zone Outdoor Air Inlet Mass Flow Rate", "key": "*", "freq": "TimeStep"},
#     {"name": "System Node Standard Density Volume Flow Rate", "key": "*", "freq": "TimeStep"},
#     # {"name":"Zone Air System Sensible Cooling Energy", "key":"SPACE2-1", "freq":"TimeStep"},
#     # {"name":"Zone Air System Sensible Cooling Energy", "key":"SPACE3-1", "freq":"TimeStep"},
#     # {"name":"Zone Air System Sensible Cooling Energy", "key":"SPACE4-1", "freq":"TimeStep"},
#     # {"name":"Zone Air System Sensible Cooling Energy", "key":"SPACE5-1", "freq":"TimeStep"},
# ], activate=True)
# 1) Make sure SQL will be produced
# util.ensure_output_sqlite(activate=True)

# 2) Variables to record (TimeStep)
specs = [
    # --- Zone state + people ---
    {"name": "Zone Mean Air Temperature",                "key": "*",            "freq": "TimeStep"},
    {"name": "Zone Mean Air Dewpoint Temperature",       "key": "*",            "freq": "TimeStep"},
    {"name": "Zone Air Relative Humidity",               "key": "*",            "freq": "TimeStep"},
    {"name": "Zone People Occupant Count",               "key": "*",            "freq": "TimeStep"},

    # --- CO₂ & OA into zones ---
    {"name": "Zone Air CO2 Concentration",               "key": "*",            "freq": "TimeStep"},
    {"name": "Zone Outdoor Air Inlet Mass Flow Rate",    "key": "*",            "freq": "TimeStep"},

    # --- Air system OA (scalar, per air loop) ---
    {"name": "Air System Outdoor Air Mass Flow Rate",    "key": "",             "freq": "TimeStep"},
    {"name": "Air System Outdoor Air Flow Fraction",     "key": "",             "freq": "TimeStep"},

    # --- Nodes (use * for all; swap to explicit node names if you prefer) ---
    {"name": "System Node Mass Flow Rate",               "key": "*",            "freq": "TimeStep"},
    {"name": "System Node Standard Density Volume Flow Rate", "key": "*",       "freq": "TimeStep"},

    # --- Infiltration (per zone) ---
    {"name": "Zone Infiltration Standard Density Volume Flow Rate", "key": "*", "freq": "TimeStep"},

    # --- Site weather (Environment key) ---
    {"name": "Site Outdoor Air Drybulb Temperature",     "key": "Environment",  "freq": "TimeStep"},
    {"name": "Site Outdoor Air Wetbulb Temperature",     "key": "Environment",  "freq": "TimeStep"},
]

# 3) Ensure the Output:Variable objects exist (dedup-aware)
util.ensure_output_variables(specs, activate=True)

util.ensure_output_sqlite()
util.verbose = 1
util.enable_runtime_logging()
# util.register_begin_iteration(["occupancy_counter"], clear=True)
util.register_begin_iteration([
    {"method_name": "occupancy_counter",
     "key_wargs": {"lam": 25.0, "min": 24, "max": 27, "seed": 123}}
], clear=True, run_during_warmup=False)
# # register JUST this one method to start
# util.register_begin_iteration([
#     {"method_name": "co2_set_outdoor_ppm_simple",
#      "key_wargs": {"value_ppm": 0.0, "log_every_minutes": 60, "verify": True}}
# ], clear=True, run_during_warmup=False)
# util.register_begin_iteration([
#     {"method_name": "co2_set_outdoor_ppm_debug",
#      "key_wargs": {"value_ppm": 0.0}}
# ], clear=True, run_during_warmup=True)  # run during warmup so you see output throughout
util.register_begin_iteration([
    {"method_name": "co2_set_outdoor_ppm",
     "key_wargs": {"value_ppm": 400.0, "log_every_minutes": None, "verify": True}}
], clear=True, run_during_warmup=False)
rc=util.run_annual()

### Table Inspect

In [ ]:
xp = EPlusSqlExplorer(f"{out_dir}/eplusout.sql")

In [ ]:
xp.list_tables()

### Analyze weather data

In [ ]:
# util.ensure_output_sqlite()
site_vars = [
    "Site Outdoor Air Drybulb Temperature",
    "Site Outdoor Air Dewpoint Temperature",
    "Site Outdoor Air Humidity Ratio",
    "Site Outdoor Air Barometric Pressure",
    "Site Wind Speed",
    "Site Wind Direction",
    "Site Diffuse Solar Radiation Rate per Area",
    "Site Direct Solar Radiation Rate per Area",
    "Site Horizontal Infrared Radiation Rate per Area",
    "Site Sky Temperature",
]
util.ensure_output_sqlite()
util.ensure_output_variables(
    [{'name': v, 'key': 'Environment', 'freq': 'TimeStep'} for v in site_vars]
)
util.run_annual()

In [ ]:
util.export_weather_sql_to_csv()

In [ ]:
weather_df=pd.read_csv('eplus_out/weather_timeseries.csv')
weather_df['timestamp'] = pd.to_datetime(weather_df['timestamp'])
weather_df['month'] = weather_df['timestamp'].dt.month

In [ ]:
weather_df

In [ ]:
import numpy as np
from scipy.stats import norm, lognorm, gamma
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Extract the temperature data
variable='Site Outdoor Air Humidity Ratio [kgWater/kgDryAir]'#'Site Outdoor Air Dewpoint Temperature [C]' #'Site Outdoor Air Drybulb Temperature [C]' #
n=9
data_df = weather_df[weather_df['month']==n]
data = data_df[variable]

# Fit a normal distribution to the data:
mu, std = norm.fit(data)

# Create the histogram trace from the previous plot
counts, bin_edges = np.histogram(data, bins=50) # Adjust bin count as needed
bin_centers = 0.5 * (bin_edges[:-1] + bin_edges[1:])

histogram_trace = go.Bar(x=bin_centers, y=counts, name='Histogram', opacity=0.7)

# Create the Gaussian curve trace
xmin, xmax = data.min(), data.max()
x_norm = np.linspace(xmin, xmax, 100)
p_norm = norm.pdf(x_norm, mu, std)

# Scale the PDF to match the histogram's count scale
bin_width = bin_edges[1] - bin_edges[0]
scaled_pdf_norm = p_norm * len(data) * bin_width

gaussian_trace = go.Scatter(x=x_norm, y=scaled_pdf_norm, mode='lines', name=f'Gaussian Fit (μ={mu:.2f}, σ={std:.2f})', line=dict(color='red', width=2))

# Fit Log-Normal distribution
# Log-normal distribution requires positive data. Since temperature can be negative,
# a simple log-normal fit might not be appropriate directly.
# However, for demonstration, we can fit it to the positive part or shift the data.
# Let's fit it to the original data, understanding the limitations if negative values exist.
# We need to be careful if temperature_data contains zero or negative values for lognorm fit.
# For simplicity, we'll add an offset if there are non-positive values.
offset = 0
if (data <= 0).any():
    offset = -data.min() + 1 # Shift data to be positive
    print(f"Shifting data by {offset:.2f} for Log-Normal fit to ensure positivity.")

shape_lognorm, loc_lognorm, scale_lognorm = lognorm.fit(data + offset)

# Generate points for the fitted Log-Normal curve
# Ensure the x range is appropriate for the shifted data
x_lognorm = np.linspace(data.min() + offset, data.max() + offset, 100)
p_lognorm = lognorm.pdf(x_lognorm, shape_lognorm, loc_lognorm, scale_lognorm)

# Scale the PDF and shift x back for plotting
scaled_pdf_lognorm = p_lognorm * len(data) * bin_width
x_lognorm_unshifted = x_lognorm - offset

lognormal_trace = go.Scatter(x=x_lognorm_unshifted, y=scaled_pdf_lognorm, mode='lines', name=f'Log-Normal Fit', line=dict(color='green', width=2))


# Fit Gamma distribution
# Gamma distribution also typically requires positive data. Similar consideration as Log-Normal.
# We'll fit it to the shifted data if an offset was applied for lognormal.
shape_gamma, loc_gamma, scale_gamma = gamma.fit(data + offset)

# Generate points for the fitted Gamma curve
# Ensure the x range is appropriate for the shifted data
x_gamma = np.linspace(data.min() + offset, data.max() + offset, 100)
p_gamma = gamma.pdf(x_gamma, shape_gamma, loc_gamma, scale_gamma)

# Scale the PDF and shift x back for plotting
scaled_pdf_gamma = p_gamma * len(data) * bin_width
x_gamma_unshifted = x_gamma - offset


gamma_trace = go.Scatter(x=x_gamma_unshifted, y=scaled_pdf_gamma, mode='lines', name=f'Gamma Fit', line=dict(color='purple', width=2))


# Create the figure and add traces
fig = go.Figure()
fig.add_trace(histogram_trace)
fig.add_trace(gaussian_trace)
fig.add_trace(lognormal_trace)
fig.add_trace(gamma_trace)


# Update layout
fig.update_layout(title=f'Distribution of {variable} with Distribution Fits',
                  xaxis_title=variable,
                  yaxis_title='Count',
                  barmode='overlay' # Overlay bars to see fits better
                 )

# Show the plot
fig.show()

### Plot Results

In [ ]:
variable="Electricity:Facility"
util.api.exchange.get_meter_handle(util.state,variable)

In [ ]:
variable = "Site Wind Direction"     # units: deg
key = 'Environment'                   # site variables use

wd_handle = {"h": -1}

def after_warmup(s):
    # ask E+ to make this variable available
    try:
        util.api.exchange.request_variable(s, variable, key)
    except Exception:
        pass
    # get the handle once
    wd_handle["h"] = util.api.exchange.get_variable_handle(s, variable, key)

def on_tick(s):
    if util.api.exchange.warmup_flag(s):
        return
    h = wd_handle["h"]
    if h != -1:
        val = util.api.exchange.get_variable_value(s, h)
        # do something with val (float degrees)
        print("Wind dir =", val)

util.reset_state()
util.api.runtime.callback_after_new_environment_warmup_complete(util.state, after_warmup)
util.api.runtime.callback_begin_system_timestep_before_predictor(util.state, on_tick)
util.api.runtime.run_energyplus(util.state, ['-w', util.epw, '-d', util.out_dir, util.idf])

In [ ]:
pd.DataFrame(util.flatten_mtd())

In [ ]:
variable="Zone Air CO2 Concentration" #"Zone Air CO2 Concentration" #"Zone Air Relative Humidity" #"Zone Air Temperature"
# 4) discover keys and plot
display(util.list_sql_zone_variables(name=variable).head(10))

zone_fig=util.plot_sql_zone_variable(
    variable,
    keys=["SPACE1-1","SPACE2-1","SPACE3-1","SPACE4-1","SPACE5-1","PLENUM-1"],
    resample="1h",
    title=f"{variable} (Hourly Mean)"
)

In [ ]:
util.plot_sql_series([
    # {"kind":"var","name":"Zone Air CO2 Concentration","key":"SPACE1-1","label":"CO2 SPACE1-1"},
    {"kind":"var","name":"Air System Outdoor Air Mass Flow Rate","key":"*","label":"OA ṁ SPACE1-1"},
    # system-level node is also helpful (replace with your OA node key if different):
    {"kind":"var","name":"System Node Mass Flow Rate","key":"*","label":"OA node V̇"},
], reporting_freq=("TimeStep","Hourly"), resample="15min", meters_to_kwh=False)

In [ ]:
# What “Zone … Outdoor Air …” style vars exist?
util.list_sql_zone_variables(like="Zone %Outdoor Air%")

# Node-based flow variables (system-level). Then skim keys that look like OA nodes.
util.list_sql_zone_variables(name="System Node Mass Flow Rate")
util.list_sql_zone_variables(name="System Node Standard Density Volume Flow Rate")

# Controller/airloop scalar:
util.list_sql_zone_variables(name="Air System Outdoor Air Flow Fraction")


In [ ]:
# generates variables_only.csv in out_dir and returns rows
rows = util.list_variables_safely(kinds="var", save_csv=True)

# quick filter in-Python for likely OA/ventilation signals
needles = ("outdoor air", "ventilation", "ideal loads outdoor air",
           "system node mass flow rate", "standard density volume flow rate",
           "zone inlet", "oa flow", "oa mass flow", "mixing box", "economizer")
cand = [r for r in rows if any(k in r["name"].lower() for k in needles)]
len(cand), sorted({r["name"] for r in cand})[:40]


In [ ]:
# Airloop scalar (look at keys to know what to plot)
util.list_sql_zone_variables(name="Air System Outdoor Air Mass Flow Rate")

# Node-based (grab OA-ish node keys)
util.list_sql_zone_variables(name="System Node Mass Flow Rate")

# Zone terminal OA (terminal names)
util.list_sql_zone_variables(name="Zone Air Terminal Outdoor Air Volume Flow Rate")

# Zone infiltration (zone names)
util.list_sql_zone_variables(name="Zone Infiltration Standard Density Volume Flow Rate")


In [ ]:
util.list_sql_zone_variables(name="Air System Outdoor Air Mass Flow Rate")

In [ ]:
# 3) See what meters actually have rows
for m in output_meteres:
    display(util.inspect_sql_meter(m, include_design_days=True))

# 4) Plot site electricity (facility)
elect_fig=util.plot_sql_meters(
    output_meteres,
    reporting_freq=("TimeStep","Hourly"),
    include_design_days=False,
    resample="1h",               # sum to hourly kWh
    meters_to_kwh=True,
    title=f"{', '.join(output_meteres)}"
)

# (optional) Net purchased if you enabled those two meters:
# elect_purchased=util.plot_sql_net_purchased_electricity(resample="1h")

In [ ]:
occ_keys = (
    util.list_sql_zone_variables(
        name='Zone People Occupant Count',
        reporting_freq=None,              # don't filter; show all
        include_design_days=False
    )['KeyValue']
    .dropna().astype(str).tolist()
)

In [ ]:
occ_keys = (
    util.list_sql_zone_variables(
        name='Zone People Occupant Count',
        reporting_freq=None,              # don't filter; show all
        include_design_days=False
    )['KeyValue']
    .dropna().astype(str).tolist()
)
occ_keys

In [ ]:
# 1) discover the exact zone keys present for the occupancy variable
occ_keys = (
    util.list_sql_zone_variables(
        name='Zone People Occupant Count',
        reporting_freq=None,              # don't filter; show all
        include_design_days=False
    )['KeyValue']
    .dropna().astype(str).tolist()
)

# 2) build selections using those keys (so keys and zones match the DB)
# output_sels = [
#     {'kind':'var','name':'Zone Mean Air Temperature','key':k,'label':f'MAT: {k}'}
#     for k in occ_keys
# ] + [
#     {'kind':'var','name':'Zone Air Relative Humidity','key':k,'label':f'ARH: {k}'}
#     for k in occ_keys
# ] +
# output_sels =  [
#     {'kind':'var','name':'Zone Air System Sensible Cooling Energy','key':k,'label':f'QSEN: {k}'}
#     for k in occ_keys
# ]
# + [
#     {'kind':'var','name':'Zone Total Internal Latent Gain Energy','key':k,'label':f'QLAT: {k}'}
#     for k in occ_keys
# ]
output_sels = [
    {'kind':'var','name':'Zone Air CO2 Concentration','key':k,'label':f'CO2: {k}'}
    for k in occ_keys
]

control_sels = (
    [{'kind':'var','name':'Zone People Occupant Count','key':k,'label':f'Occ: {k}'} for k in occ_keys]
    # + [
    #     {'kind':'var','name':'Site Outdoor Air Drybulb Temperature','key':'Environment','label':'OAT'},
    #     {'kind':'var','name':'Site Outdoor Air Wetbulb Temperature','key':'Environment','label':'OWB'},
    # ]
)

# 3) plot covariance (pull any freq; we resample to 1H anyway)
fig = util.plot_sql_cov_heatmap(
    control_sels=control_sels,
    output_sels=output_sels,
    reporting_freq=None,     # <- don't filter out Zone Timestep rows
    resample='1h',           # compute cov on hourly series
    reduce='mean',
    stat='cov',              # or 'corr' if you want scale-free
    min_periods=12,
    include_design_days=False
)

In [ ]:
occ_keys

In [ ]:
# 1) discover zone keys that exist for the variable
occ_keys = (
    util.list_sql_zone_variables(
        name='Zone People Occupant Count',
        reporting_freq=None,            # don't filter; accept Zone Timestep, Hourly, etc.
        include_design_days=False
    )['KeyValue']
    .dropna().astype(str).tolist()
)

# (optional) limit to first N zones
# occ_keys = occ_keys[:8]

# 2) build selections and plot
selections = [
    {'kind':'var', 'name':'Zone People Occupant Count', 'key':k, 'label':k}
    for k in occ_keys
]

fig = util.plot_sql_series(
    selections=selections,
    reporting_freq=None,      # pull whatever is in the DB
    resample='1h',            # average to hourly; set to None for native timestep
    aggregate_vars='mean',    # hourly mean occupancy; use 'sum' for person-hours per hour
    title='Occupant Count per Zone',
    show=True
)

In [ ]:
fig = util.plot_sql_series(
    selections=output_sels,
    reporting_freq=None,      # pull whatever is in the DB
    resample='1h',            # average to hourly; set to None for native timestep
    aggregate_vars='mean',    # hourly mean occupancy; use 'sum' for person-hours per hour
    title='Occupant Count per Zone',
    show=True
)

# SQLITE - Table Inspect

In [ ]:
xp.list_tables()

# Documentation

https://energyplus.net/assets/nrel_custom/pdfs/pdfs_v25.1.0/EngineeringReference.pdf

https://energyplus.net/assets/nrel_custom/pdfs/pdfs_v24.1.0/InputOutputReference.pdf

# Sri Lanka Weather Data

https://climate.onebuilding.org/WMO_Region_2_Asia/LKA_Sri_Lanka/index.html

#HVAC Design

* https://www.mdpi.com/1996-1073/16/20/7124
* https://www.mdpi.com/2071-1050/17/5/1955
* https://discovery.ucl.ac.uk/id/eprint/10116413/1/manuscript%20baycal.pdf
* https://arxiv.org/pdf/2508.09118#:~:text=Page%202,(7)
* https://link.springer.com/article/10.1007/s12273-025-1300-4
* https://www.mdpi.com/2075-5309/13/2/314?
* (Bayes) https://www.sciencedirect.com/science/article/pii/S0306261925013017

A compact state-space model for a single zone with temperature and moisture dynamics, plus an output map that reports space dry-bulb and space RH.


Definitions (symbols & constants)

States:
\begin{align}
x &= \begin{bmatrix}T_z\\ \omega_z\\ c_z\end{bmatrix}
\end{align}
where

* $T_z$ is room dry-bulb [°C],
* $\omega_z$ is the humidity ratio [$kg/kg_{dry}$]
* $c_z$ is the $CO_2$ concentration [$kg/kg_{dry}$]

Control inputs:
\begin{align}
u_1 &= m_{sa} \:\textrm{(supply dry-air mass flow [kg/s])},\\
u_2 &= T_{sa}, \\
u_3 &= \omega_{sa}
\end{align}

Disturbances:


* $T_o, \omega_o, c_o$ (outdoor),
* $m_{oa}$ (OA+infiltration dry-air flow),
* $Q_{bg}$ (sensible) [W],
* $G_{bg}$ (moisture) [kg/s],
* $q^{occ}_{sens}$ [W/person] is per-person sensible heat and $f_c\in[0,1]$ is the convective fraction (radiant part mainly warms surfaces; taking $f_c\approx 0.6$ is common if you don't model wall states).
* $g^{occ}_{v}$ [kg/s/person] the per-person vapor generation.
* $ g_{CO2}^{occ} $ per-person CO₂ generation rate [kg/s/person]

Constants:

* $C_s$ effective sensible capacitance [J/K],
* $M_m = \rho_{air}V$ moisture capacity [kg{dry}],
* $UA$ [W/K],
* $c_{pa}\approx1006 J/(kg·K)$

⸻

Governing dynamics (from energy & moisture balances)

Define gains
$k_U=\frac{UA}{C_s},\quad k_{sa}=\frac{c_{pa}}{C_s},\quad k_m=\frac{1}{M}$.

Then the scalar dynamics are
\begin{align}
\dot T_z
&= -\,k_U\,T_z \;-\; k_{sa}\,(m_{oa}+m_{sa})\,T_z
\;+\; k_U\,T_o \;+\; k_{sa}\,m_{oa}\,T_o \;+\; k_{sa}\,m_{sa}\,T_{sa}
\;+\; \frac{Q_{bg}}{C_s}\;+\;\frac{f_c\,q^{occ}_{sens}}{C_s}N,
\\
\dot \omega_z
&= -\,k_m\,(m_{oa}+m_{sa})\,\omega_z
\;+\; k_m\,m_{oa}\,\omega_o \;+\; k_m\,m_{sa}\,\omega_{sa}
\;+\; \frac{G_{bg}}{M}
\;+\; \frac{g^{occ}_{v}}{M}\,N.\\
\dot c_z
&= -k_m\,(m_{oa}+m_{sa})\,c_z
\;+\; k_m\,m_{oa}\,c_o
\;+\; k_m\,m_{sa}\,c_{sa}
\;+\; \frac{g_{CO2}^{occ}}{M}\,N
\end{align}


These equations are bilinear (flows $\times$ states) and affine in the exogenous terms.

⸻

Compact state–space (bilinear/affine form)

Let
\begin{align}
A_0&=\begin{bmatrix}-k_U&0&0\\0&0 &0\\
0 & 0 & 0\end{bmatrix},\\
A_1&=\begin{bmatrix}-k_{sa}&0&0\\0&-k_m&0\\0 & 0 & -k_m\end{bmatrix},
\end{align}
and define the “composite inputs”
\begin{align}
\phi &=\ \begin{bmatrix}
m_{sa}T_{sa} \\ m_{oa}T_o \\ m_{sa}\omega_{sa} \\ m_{oa}\omega_o
\\ m_{sa}c_{sa} \\ m_{oa}c_o \\ Q_{bg} \\ G_{bg}
\end{bmatrix}.
\end{align}

\begin{align}
B_o&=\begin{bmatrix}
\frac{f_c\,q^{occ}_{sens}}{C_s}\\
\frac{g^{occ}_{v}}{M}\\
\frac{g^{occ}_{CO2}}{M}
\end{bmatrix}
\end{align}
Then
\begin{align}
\dot{x} &= A_0\,x \;+\; (m_{sa}+m_{oa})\;A_1\,x
\;+\; D\,\begin{bmatrix}T_o\\ \omega_o\\ c_o\end{bmatrix}
\;+\; B_u\,\phi\;+\;B_oN
\end{align}
with
\begin{align}
D&=\begin{bmatrix}k_U&0&0\\0&0&0\\0&0&0\end{bmatrix},\\
B_u&=\begin{bmatrix}
k_{sa} & k_{sa} & 0 & 0 & 0 & 0 & \tfrac{1}{C_s} & 0\\
0 & 0 & k_m & k_m & 0 & 0 & 0 & \tfrac{1}{M}\\
0 & 0 & 0 & 0 & k_m & k_m & 0 & 0
\end{bmatrix}.
\end{align}

⸻

Output map (room DB and RH)

We report
\begin{align}
y&=\begin{bmatrix}y_1\\y_2\\ y_3\end{bmatrix}
=\begin{bmatrix}T_z\\ \mathrm{RH}z\\c_z\end{bmatrix}.
\end{align}
The RH is a nonlinear function of $T_z,\omega_z$:
\begin{align}
\mathrm{RH}z &=\; \frac{p_v(T_z,\omega_z)}{p_{ws}(T_z)},\\
p_v &=\frac{\omega_z\,p_{atm}}{0.62198+\omega_z},
\end{align}
with saturation vapor pressure (Tetens, $T_z$ in °C, Pa)
\begin{align}
p_{ws}(T_z) &= 610.94 \,\exp\left(\frac{17.625\,T_z}{T_z+243.04}\right).
\end{align}

Equivalently, in measurement form
\begin{align}
y_1 &= [\,1\;\;0\,]\,x,\\
y_2 &= \frac{\displaystyle \frac{\omega_z\,p_{atm}}{0.62198+\omega_z}}
{610.94 \exp\left(\frac{17.625\,T_z}{T_z+243.04}\right)}.
\end{align}

⸻

Notes for control/estimation
* Treat $(m_{sa},m_{oa})$ as scheduling variables (LPV) or keep the bilinear form for EKF/UKF design.
* If you prefer a strictly linear form, you can hold $(m_{sa},m_{oa})$ at an operating point and linearize (get A,B about that point).
* The same structure extends to a 2R2C thermal model by adding an envelope state $T_w$ and splitting UA across $T_o\leftrightarrow T_w\leftrightarrow T_z$.

---

\begin{align}
T_z^\star
&=\frac{UA\,T_o\;+\;c_{pa}\,m_{\inf}\,T_o\;+\;c_{pa}\,m_{sa}\,T_{sa}\;+\;Q_{bg}\;+\;f_c\,q^{occ}_{sens}N}
{UA\;+\;c_{pa}\,(m_{sa}+m_{\inf})}\\
\omega_z^\star
&=\frac{m_{\inf}\,\omega_o\;+\;m_{sa}\,\omega_{sa}\;+\;G_{bg}\;+\;g^{occ}_{v}N}
{m_{sa}+m_{\inf}}\\
c_z^\star
&=\frac{m_{\inf}\,c_o\;+\;m_{sa}\,c_{sa}\;+\;g^{occ}_{CO2}N}
{m_{sa}+m_{\inf}}
\end{align}

---

\begin{align}
T_{sa} - T_z^\star &= \frac{-[UA + c_{pa} m_{\inf}](T_z^\star  - T_o) + Q_{bg} + f_c q^{occ}_{sens}N}{c_{pa} m_{sa}} \\
\omega_{sa} - \omega_z^\star &= \frac{m_{\inf}\left(\omega_z^\star-\omega_o\right) - G_{bg} - g_v^{occ}N}{m_{sa}} \\
c_{sa} - c_z^\star &= \frac{m_{\inf}\left(c_z^\star-c_o\right) - g_{CO2}^{occ}N}{m_{sa}}
\end{align}

\begin{align*} T_z^\star - T_o &= \frac{Q_{bg} + f_c q^{occ}_{sens}N - c_{pa} m_{sa} (T_{sa} - T_z^\star)}{UA + c_{pa} m_{\inf}} \\ \omega_z^\star - \omega_o &= \frac{m_{sa}(\omega_{sa} - \omega_z^\star) + G_{bg} + g_v^{occ}N}{m_{\inf}} \\
c_z^\star - c_o &= \frac{m_{sa}(c_{sa} - c_z^\star) + g_{CO2}^{occ}N}{m_{\inf}} \end{align*}

## Uncertainty Model-0

\begin{align}
T_z^\star
&=\frac{UA\,T_o\;+\;c_{pa}\,m_{\inf}\,T_o\;+\;c_{pa}\,m_{sa}\,T_{sa}\;+\;Q_{bg}\;+\;f_c\,q^{occ}_{sens}N}
{UA\;+\;c_{pa}\,(m_{sa}+m_{\inf})}\\
\omega_z^\star
&=\frac{m_{\inf}\,\omega_o\;+\;m_{sa}\,\omega_{sa}\;+\;G_{bg}\;+\;g^{occ}_{v}N}
{m_{sa}+m_{\inf}}\\
c_z^\star
&=\frac{m_{\inf}\,c_o\;+\;m_{sa}\,c_{sa}\;+\;g^{occ}_{CO2}N}
{m_{sa}+m_{\inf}}
\end{align}

Let
\begin{align}
\theta_T& =\frac{UA+{c_{pa}}m_{\inf}}{UA+{c_{pa}}(m_{sa}+m_{\inf})}\\
\theta_m & =\frac{m_{inf}}{m_{sa}+m_{inf}}
\end{align}

\begin{align}
T_z^\star
&=\theta_T\,T_o\;+\;(1-\theta_T)\,T_{sa}\;+\;\frac{Q_{bg}\;+\;f_c\,q^{occ}_{sens}N}
{UA\;+\;c_{pa}\,(m_{sa}+m_{\inf})}\\
\omega_z^\star
&=\theta_m\,\omega_o\;+\;(1-\theta_m)\,\omega_{sa}\;+\;\frac{G_{bg}\;+\;g^{occ}_{v}N}
{m_{sa}+m_{\inf}}\\
c_z^\star
&=\theta_m\,c_o\;+\;(1-\theta_m)\,c_{sa}\;+\;\frac{g^{occ}_{CO2}N}
{m_{sa}+m_{\inf}}
\end{align}

\begin{align}
T_z^\star
&=\theta_T\,(T_o-T_{sa})\;+\;T_{sa}\;+\;\frac{Q_{bg}\;+\;f_c\,q^{occ}_{sens}N}{UA\;+\;c_{pa}\,(m_{sa}+m_{\inf})}\\
\omega_z^\star
&=\theta_m\,(\omega_o-\omega_{sa})\;+\;\omega_{sa}\;+\;\frac{G_{bg}\;+\;g^{occ}_{v}N}{m_{sa}+m_{\inf}}\\
c_z^\star
&=\theta_m\,(c_o-c_{sa})\;+\;c_{sa}\;+\;\frac{g^{occ}_{CO2}N}
{m_{sa}+m_{\inf}}
\end{align}

---

Let
\begin{align}
y\triangleq &=\begin{bmatrix}
y_T \\   y_{\omega}\\ y_{CO2}
\end{bmatrix}
= \begin{bmatrix}
T_z^\star\\
\omega_z^\star\\
c_z^\star
\end{bmatrix}
\end{align}

---

\begin{align}
\phi &= \begin{bmatrix}
(T_o - T_{sa}) & 1 & 0 & 0 & 0 \\
0 & 0 & (\omega_o - \omega_{sa}) & 1 &0 \\
0 & 0 & (c_o - c_{sa}) & 0 & 1\\
\end{bmatrix}\\
\beta &= \begin{bmatrix}
\theta_T \\
T_{sa}+\frac{Q_{bg}\;+\;f_c\,q^{occ}_{sens}N}{UA\;+\;c_{pa}\,(m_{sa}+m_{\inf})}\\
\theta_m\\
\omega_{sa}\;+\;\frac{G_{bg}\;+\;g^{occ}_{v}N}{m_{sa}+m_{\inf}}\\
c_{sa}\;+\;\frac{g^{occ}_{CO2}N}
{m_{sa}+m_{\inf}}
\end{bmatrix}
\end{align}

---

\begin{align}
y = \phi \beta + \epsilon, \qquad \epsilon \sim \mathscr{N}(0,\Sigma_R)
\end{align}

## Uncertainty Model-1

Let the **measured set** be
$$\mathscr M = \{ T_z^\star,\ \omega_z^\star,\ c_z^\star,\ T_o,\ \omega_o,\ c_o,\ m_{sa} \},$$
and define the (unknown) lumped terms
$$\theta_{inf}=m_{\inf},$$
$$\theta_{T}={UA+c_{pa}m_{\inf}}=UA+{c_{pa}}m_{\inf}.$$

Then your steady-state inverses become


\begin{align}
m_{sa}(c_{sa}-c_z^\star)
&=
\theta_{inf}\left(c_z^\star-c_o\right)
-g_{CO2}^{occ}N_o\\
m_{sa}(\omega_{sa}-\omega_z^\star)
&=
\theta_{inf}\left(\omega_z^\star-\omega_o\right)
-G_{bg}-g_v^{occ}N_o\\
c_{pa}m_{sa}(T_{sa}- T_z^\star)
&=
\theta_T\left(T_z^\star-T_o\right)
-Q_{bg}-f_c q^{occ}_{sens}N_o
\end{align}

---

Let
\begin{align}
y_T\triangleq c_{pa}m_{sa}(T_{sa}- T_z^\star),\\
y_{\omega}\triangleq m_{sa}(\omega_{sa}- \omega_z^\star),\\
y_{CO2}\triangleq m_{sa}(c_{sa}- c_z^\star),
\end{align}
\begin{align}
x_T\triangleq (T_z^\star-T_o),\\
x_{\omega}\triangleq (\omega_z^\star-\omega_{o}),\\
x_{CO2}\triangleq (c_z^\star-c_{o})
\end{align}
\begin{align}
y_{CO2}
&=
\theta_{inf}x_{CO2}
-g_{CO2}^{occ}N_o\\
y_{\omega}
&=
\theta_{inf}x_{\omega}
-G_{bg}-g_v^{occ}N_o\\
y_T
&=
\theta_{T}x_T
-Q_{bg}-f_c q^{occ}_{sens}N_o
\end{align}

* $X=[x_T\:\:\:x_\omega:\:\:x_{CO2}]^T\sim \mathscr{N}\left(\mu,\Sigma\right)$
* $N_o \sim \Gamma(\alpha, \beta)$
* $\Theta = \frac{1}{m_{sa}}\mathrm{diag}\left([\theta_T\:\:\:\theta_{inf}]\right) \sim \Gamma(\alpha, \beta)$

## Uncertainty Model-2

Let the **measured set** be
$$\mathscr M = \{ T_z^\star,\ \omega_z^\star,\ c_z^\star,\ T_o,\ \omega_o,\ c_o,\ m_{sa} \},$$
and define the (unknown) lumped terms
$$\theta_{inf}=\frac{m_{\inf}}{m_{sa}},$$
$$\theta_{T}=\frac{UA+{c_{pa}}m_{\inf}}{c_{pa}m_{\inf}}.$$

Then your steady-state inverses become


\begin{align}
m_{sa}(c_{sa}-c_z^\star)
&=
\theta_{inf}\left(c_z^\star-c_o\right)
-g_{CO2}^{occ}N_o\\
m_{sa}(\omega_{sa}-\omega_z^\star)
&=
\theta_{inf}\left(\omega_z^\star-\omega_o\right)
-G_{bg}-g_v^{occ}N_o\\
c_{pa}m_{sa}(T_{sa}- T_z^\star)
&=
\theta_T\left(T_z^\star-T_o\right)
-Q_{bg}-f_c q^{occ}_{sens}N_o
\end{align}

---

Let
\begin{align}
y_T\triangleq (T_{sa}- T_z^\star),\\
y_{\omega}\triangleq (\omega_{sa}- \omega_z^\star),\\
y_{CO2}\triangleq (c_{sa}- c_z^\star),
\end{align}
\begin{align}
x_T\triangleq (T_z^\star-T_o),\\
x_{\omega}\triangleq (\omega_z^\star-\omega_{o}),\\
x_{CO2}\triangleq (c_z^\star-c_{o})
\end{align}
\begin{align}
y_{CO2}
&=
\theta_{inf}x_{CO2}
-g_{CO2}^{occ}N_o\\
y_{\omega}
&=
\theta_{inf}x_{\omega}
-G_{bg}-g_v^{occ}N_o\\
y_T
&=
\theta_{T}x_T
-Q_{bg}-f_c q^{occ}_{sens}N_o
\end{align}

* $X=[x_T\:\:\:x_\omega:\:\:x_{CO2}]^T\sim \mathscr{N}\left(\mu,\Sigma\right)$
* $N_o \sim \Gamma(\alpha, \beta)$
* $\Theta = \frac{1}{m_{sa}}\mathrm{diag}\left([\theta_T\:\:\:\theta_{inf}]\right) \sim \Gamma(\alpha, \beta)$

Totally doable. The cleanest way is to cast your two equations as a **Bayesian linear regression** (stacked over time) and then choose either a **conjugate Gaussian** update (simple, closed-form) or a **positivity-preserving** variant (log-parameters + EKF/MCMC). Below I give both.

---

# 1) Put the model in regression form

For each time (k) define
\begin{align}
x_{T,k}&\triangleq (T_{z,k}^\star-T_{o,k}),\\
x_{\omega,k}&\triangleq (\omega_{z,k}^\star-\omega_{o,k}),\\
x_{CO2,k}&\triangleq (c_{z,k}^\star-c_{o,k}),
\end{align}
\begin{align}
y_k &\equiv
\begin{bmatrix}
y_{T,k}\\y_{\omega,k}\\y_{CO2,k}
\end{bmatrix}
\equiv
\begin{bmatrix}
m_{sa,k}c_{pa}(T_{sa,k}-T_{z,k}^\star)\\
m_{sa,k}(\omega_{sa,k}-\omega_{z,k}^\star)\\
m_{sa,k}(c_{sa,k}-c_{z,k}^\star)
\end{bmatrix},\\
\phi_k &\equiv
\begin{bmatrix}
x_{T,k} & -1 & 0 & 0\\
0 & 0 & x_{\omega,k} & -1\\
0 & 0 & x_{CO2,k} & 0
\end{bmatrix},\\
\psi_k &\equiv
\begin{bmatrix}
f_c q^{occ}_{sens}\\
g_v^{occ}\\
g_{CO2}^{occ}
\end{bmatrix},
\end{align}


\begin{align}
\beta &\equiv
\begin{bmatrix}
\theta_T\\
Q_{bg}\\
\theta_{inf}\\
G_{bg}
\end{bmatrix}
\end{align}


Then the measurement model is simply
\begin{align}
y_k &= \phi_k\beta + \psi_k N_o +\varepsilon_k,\qquad
\varepsilon_k \sim \mathscr{N}(0,R),
\end{align}
where $(R=\operatorname{diag}(\sigma_T^2,\ \sigma_\omega^2))$ are the (unknown) error variances for the two equations.

---

Stack (n) samples:
\begin{align}
y &= \Phi\beta + \varepsilon,\\
y\in\mathbb R^{2n},\\
\Phi\in\mathbb R^{2n\times 6},\\
\varepsilon\sim\mathscr{N}(0,;I_n\otimes R).
\end{align}


---

# 2) Conjugate Bayes update (fast & closed-form)

Choose priors:
\begin{align}
\beta \sim \mathscr{N}(\mu_0,\Sigma_0),\\
\sigma_T^2 \sim \operatorname{InvGamma}(a_T,b_T),\\
\sigma_\omega^2 \sim \operatorname{InvGamma}(a_\omega,b_\omega),
\end{align}
with $(\mu_0,\Sigma_0)$ weakly informative (e.g., large variances; center $(\theta_T,\theta_\omega)$ positive).

Compute block-diagonal $(R=\operatorname{diag}(\sigma_T^2,\sigma_\omega^2))$ and $(W\equiv I_n\otimes R^{-1})$.

**Posterior of $(\beta\mid\sigma_T^2,\sigma_\omega^2)$:**
\begin{align}
\Sigma_n &= \big(\Sigma_0^{-1}+\Phi^\top W,\Phi\big)^{-1},\\
\mu_n = \Sigma_n\big(\Sigma_0^{-1}\mu_0+\Phi^\top W,y\big).
\end{align}

**Posterior of variances (independent per channel):**
Let residuals per channel be $(e_T = y_T-\Phi_T\beta), (e_\omega=y_\omega-\Phi_\omega\beta)$.
Then
\begin{align}
\sigma_T^2 \mid \beta,\text{data}\ \sim\ \operatorname{InvGamma}\Big(a_T+\tfrac{n}{2},\ b_T+\tfrac{1}{2},e_T^\top e_T\Big),\\
\sigma_\omega^2 \mid \beta,\text{data}\ \sim\ \operatorname{InvGamma}\Big(a_\omega+\tfrac{n}{2},\ b_\omega+\tfrac{1}{2},e_\omega^\top e_\omega\Big).
\end{align}

**Gibbs sampler (few lines):**

1. Initialize $((\beta,\sigma_T^2,\sigma_\omega^2))$.
2. Sample $(\beta \sim \mathscr{N}(\mu_n,\Sigma_n))$ given current variances.
3. Sample $(\sigma_T^2,\sigma_\omega^2)$ from the inverse-gamma posteriors given $(\beta)$.
4. Repeat; summarize $(\beta)$ posteriors (means/intervals) and predictive uncertainty.

This gives you full posteriors **analytically** (no HMC needed), runs in milliseconds, and is easy to make **online** via a **recursive** (Kalman/RLS) version by taking $(\beta)$ as a static state with tiny process noise.


---

# 3) Positivity-preserving option (log-parameters)

If you insist on $(\theta_T,\theta_\omega,Q_{bg},G_{bg},f_cq^{occ}_{sens},g_v^{occ}\ge 0)$, re-parameterize
\begin{align}
\beta_j=\exp(\eta_j),\qquad \eta\sim \mathscr{N}(m_0,S_0).
\end{align}
Now the model is **nonlinear** in $(\eta)$:

\begin{align}
y_k = \phi_k\exp(\eta) + \varepsilon_k.
\end{align}
Do Bayes update via:

* **Laplace approximation** (one Newton step per batch), or
* **UKF** treating $(\eta)$ as the state (static) with small process noise, or
* **HMC/NUTS** (PyMC/NumPyro) with a Normal prior on (\eta).

This keeps all parameters strictly positive and is robust when you later fold in mild uncertainty on $(x)$ or $(N_o)$.

---

# 4) Online (sequential) update = “Kalman for coefficients”

Treat $(\beta_{k+1}=\beta_k+w_k)$ with $(w_k\sim\mathscr N(0,Q_\beta))$ (small), and the measurement
\begin{align}
y_k = \phi_k\beta_k + \varepsilon_k.
\end{align}
Then standard KF/RLS recursions give **running posteriors** $((\mu_k,\Sigma_k))$ for $(\beta)$:
\begin{align}
&\text{Predict: } \mu^-=\mu_{k-1},\ \Sigma^-=\Sigma_{k-1}+Q_\beta\\
&\text{Gain: } K=\Sigma^- \phi_k^\top(\phi_k\Sigma^- \phi_k^\top+R)^{-1}\\
&\text{Update: } \mu_k=\mu^- + K(y_k-\phi_k\mu^-),\ \Sigma_k=(I-K\phi_k)\Sigma^-.
\end{align}
This is the **Bayesian update** in real time, converging to the batch posterior as (k) grows.

---

# 5) Using your “Gamma priors” idea

You suggested $(\Theta\sim \Gamma(\alpha,\beta))$. With a **Gaussian likelihood**, Gamma on linear coefficients is **non-conjugate**. You can still do it via:

* **Metropolis-within-Gibbs**: sample the positive coefficients from their Gamma priors with a log-posterior proposal, keeping the Gaussian conditionals for the rest; or
* **Transform** to logs and use Gaussian priors on $(\eta=\log\beta)$ (Section 3), which behaves similarly but is easier to implement and tune.

---

## What you’ll get out

* Posterior means/intervals for $(\theta_T,\theta_\omega,Q_{bg},G_{bg},f_c q^{occ}_{sens},g_v^{occ})$.
* Predictive distributions for $(T_{sa},\omega_{sa})$ (and hence for coil loads / energy).
* If you run the **online** version, you'll have continuously “self-calibrated” coefficients that adapt to seasonal changes.

If you want, I can drop a 30-line Python (NumPy/PyMC) snippet for either the **conjugate Gibbs** or the **log-parameter Laplace** update that you can paste into your trial.


## Bayes Model

Then your measurement model is simply
\begin{align}
y_k &= \phi_k\beta + \psi_k N_o +\varepsilon_k,\qquad
\varepsilon_k \sim \mathscr{N}(0,R),
\end{align}
where $(R=\operatorname{diag}(\sigma_T^2,\ \sigma_\omega^2,\ \sigma_c^2))$ are the (unknown) error variances for the two equations.

\begin{align}
\mathscr{P}\left(\beta,N_o|y\right)\propto  \mathscr{P}\left(y|\beta,N_o\right)\mathscr{P}\left(\beta,N_o\right)
\end{align}

Yes—your “two-step Bayes” (first infer (N), then estimate (\beta) using (\mathbb E[N])) is well-aligned with current practice. You can justify it with:

## 1) Bayesian occupancy inference (Step 1)

* **Bayesian filtering of occupancy from CO₂.** Jiang et al. (Energy & Buildings): Bayesian filter combining a stochastic occupancy model (time-varying Markov / Poisson) with a CO₂ mass-balance observation model; delivers a full posterior (p(N_k\mid \text{data})). Use their setup to support your Poisson prior and per-tick Bayesian update for (N). ([Aalto University's research portal][1])
* **CO₂ “sensing by proxy,” Bayesian/MCMC.** Jin & Bayen show Bayesian inference of occupancy/emission rates from CO₂ with a PDE–ODE model; they explicitly compute posteriors for occupancy-related quantities—ideal precedent for taking (\mathbb E[N_k]) as a plug-in regressor. ([bayen.berkeley.edu][2])
* **Poisson/NHPP occupancy as a standard count model.** Reviews and assessments document Poisson/non-homogeneous Poisson processes as canonical for arrival/counts in offices; use this to motivate your Poisson prior on (N_k). ([ScienceDirect][3])

## 2) Bayesian estimation of thermal/moisture parameters (Step 2)

* **Bayesian calibration of building thermal models.** Multiple papers calibrate lumped parameters (UA, gains, etc.) by Bayesian linear/Gaussian likelihoods—exactly your (\beta) block—so plugging (\mathbb E[N_k]) into the design matrix is a standard “errors-in-inputs” simplification. Cite Calama-González et al. (Applied Energy 2021), Cui et al. (Buildings 2022), and Kristensen et al. (Energy Procedia 2017). ([ScienceDirect][4])

## 3) Using estimated parameter distributions in comfort–energy optimization

* **Chance-constrained / uncertainty-aware MPC.** A recent framework takes parameter/posterior uncertainty from identification and embeds it in a chance-constrained optimizer—exactly the link from your Bayesian estimation to comfort-zone & energy objectives. Use this to justify propagating (\mathrm{Var}(\beta)) (and (\mathrm{Var}(N)), if desired) into tightened constraints for (T_z,\omega_z). ([IBPSA Publications][5])
* **Humidity-aware MPC field work.** While not Bayesian on (T_{sa},\omega_{sa}), it is a near-term precedent for the second step (optimize comfort & energy with explicit humidity constraints), which your Bayesian estimates would feed. ([ResearchGate][6])

## How to phrase it in your paper

> *We adopt a two-stage Bayesian procedure. First, we infer the posterior of occupancy (p(N_k\mid \text{CO₂},\text{meteo})) using a Poisson/Markov prior and a CO₂ mass-balance likelihood, following Bayesian occupancy estimators in the literature.* ([Aalto University's research portal][1])
> *Second, conditioning on (\mathbb E[N_k]), we perform Bayesian linear regression to estimate lumped sensible/latent parameters (\beta) in our steady-state inverse model, consistent with standard Bayesian calibration of building thermal models.* ([ScienceDirect][4])
> *Finally, we embed the identified parameter distributions in a chance-constrained optimizer to keep ((T_z,\omega_z)\in D) while minimizing energy, as in recent uncertainty-aware HVAC MPC.* ([IBPSA Publications][5])

If you want, I can draft a short “Methods” subsection that (i) writes the Poisson-Bayes update for (N_k), (ii) the conditional Gaussian posterior for (\beta\mid \mathbb E[N]), and (iii) the chance-constraint reformulation you’ll use in the controller.

[1]: https://research.aalto.fi/files/38353202/ENG_Jiang_Bayesian_filtering_EB.pdf?utm_source=chatgpt.com "Bayesian filtering for building occupancy estimation from ..."
[2]: https://bayen.berkeley.edu/sites/default/files/sensing_by_proxy.pdf?utm_source=chatgpt.com "Sensing by Proxy: Occupancy Detection Based on Indoor CO2 ..."
[3]: https://www.sciencedirect.com/science/article/abs/pii/S0378778821003893?utm_source=chatgpt.com "Assessment of probabilistic models to estimate the ..."
[4]: https://www.sciencedirect.com/science/article/abs/pii/S0306261920315361?utm_source=chatgpt.com "Bayesian calibration of building energy models for ..."
[5]: https://publications.ibpsa.org/proceedings/asim/2024/papers/E10_asim2024_1109.pdf?utm_source=chatgpt.com "Chance-constraint MPC approach to optimize ..."
[6]: https://www.researchgate.net/publication/381921834_Humidity-Aware_Model_Predictive_Control_for_Residential_Air_Conditioning_A_Field_Study?utm_source=chatgpt.com "(PDF) Humidity-Aware Model Predictive Control for ..."


##Modelling and Control

Themeasurement model is simply
\begin{align}
y_k &= \phi_k\beta + \varepsilon_k,\qquad
\varepsilon_k \sim \mathscr{N}(0,\Sigma_R),
\end{align}
where $(R=\operatorname{diag}(\sigma_T^2,\ \sigma_\omega^2,\ \sigma_c^2))$ are the (unknown) error variances for the three equations.

\begin{align}
\beta_{k} & = \beta_{k-1}+ w_k ,\qquad
w_k \sim \mathscr{N}(0,\Sigma_P),\\
y_k &= \phi_k \beta_k  +\varepsilon_k,\qquad
\varepsilon_k \sim \mathscr{N}(0,\Sigma_R),
\end{align}
where $(\Sigma_R=\operatorname{diag}(\sigma_T^2,\ \sigma_\omega^2,\ \sigma_c^2))$ are the (unknown) error variances for the three equations.

---

The Kalman filter
\begin{align}
&\text{Predict: } \mu^-=\mu_{k-1},\ \Sigma^-=\Sigma_{k-1}+\Sigma_P\\
&\text{Gain: } K_k=\Sigma^- \phi_k^\top(H_k\Sigma^- \phi_k^\top+\Sigma_R)^{-1}\\
&\text{Update: } \mu_k=\mu^- + K_k(y_k-\phi_k\mu^-),\ \Sigma_k=(I-K_k\phi_k)\Sigma^-.
\end{align}

---

\begin{align}
\widehat{y}_k &\triangleq \phi_k\mu_k
\end{align}

Let
\begin{align}
\mu_k =\begin{bmatrix}\widehat{T}^\star_{z,k}\\ \widehat{\omega}^\star_{z,k}\\\widehat{c}^\star_{z,k}
\end{bmatrix}
\end{align}

Here are generalized expressions ** for an HVAC system with **air mixing, cooling/heating coil, and reheat**:

***

### 1. **Air Mixing (Pre-coil)**
Assume the air handler mixes **outdoor air (OA)** and **return air (RA)**:
- $ m_{oa,k} $: outdoor air mass flow
- $ m_{ra,k} = m_{sa,k-1}$: return air mass flow
- $ m_{ma,k} = m_{oa,k} +m_{ra,k}$: total mixed air mass flow

The **mixed air properties** entering the coil are:

\begin{align}
T_{ma,k} &= \frac{m_{oa,k} T_{o,k} + m_{ra,k} T^\star_{z,k}}{m_{ma,k}},\\
\omega_{ma,k} &= \frac{m_{oa,k}\omega_{o,k} + m_{ra,k}\omega^\star_{z,k}}{m_{ma,k}},\\
c_{m,k} &= \frac{m_{oa,k}c_{o,k} + m_{ra,k}c^\star_{z,k}}{m_{ma,k}}
\end{align}

Let $\gamma_{m,k}=\frac{m_{o,k}}{m_{oa,k}+m_{ra,k}}$ then
\begin{align}
T_{ma,k} &= m_{ma,k}\left(\gamma_{m,k} T_{o,k} + (1-\gamma_{m,k}) T^\star_{z,k}\right),\\
\omega_{ma,k} &= m_{ma,k}\left(\gamma_{m,k}\omega_{o,k} + (1-\gamma_{m,k})\omega^\star_{z,k}\right),\\
c_{m,k} &= m_{ma,k}\left(\gamma_{m,k}c_{o,k} + (1-\gamma_{m,k})c^\star_{z,k}\right)
\end{align}



### Sensible (Thermal) Energy Required by the Coil

For each time step, the energy rate required to move the mixed air from **mixed condition ($T_{ma,k}$)** to the **coil outlet or supply air temperature (${T}_{sa,k+1}$)** is:
\begin{align}
Q_{\text{coil},k+1} = {m}_{ma,k} \, c_{pa} |({T}_{\text{sa},k+1} - T_{\text{ma},k})|
\end{align}
- ${T}_{\text{sa},k+1}$: supply air temperature [°C] for the next step
- $T_{\text{ma},k}$: mixed air temperature entering coil [°C]

***

### Latent (Moisture) Energy Cost

If moisture is **removed** (dehumidification by cooling coil):
\begin{align}
Q_{\text{latent},k} &= {m}_{ma,k} \cdot h_{fg} \cdot \max(0, \omega_{\text{ma},k} - {\omega}^\star_{z,k+1})
\end{align}

- $h_{fg}$: latent heat of vaporization (≈ 2,500,000 J/kg at room temperature)
- $\omega_{\text{ma},k}$: mixed air humidity ratio (kg water/kg dry air)
- ${\omega}_{\text{sa},k+1}$: supply air humidity ratio at the next step

(Coils don't **add** moisture, so this term is only positive when $\omega_{sa,k} < \omega_{ma,k+1}$.)

***

### Combined Coil Power (Cooling or Heating)

The **total coil load at time $k$** (assuming no reheat for simplicity) is:
\begin{align}
Q_{\text{coil,total},k+1} &= Q_{\text{coil},k+1} + Q_{\text{latent},k+1},\\
&= {m}_{ma,k} \left[c_{pa}|({T}_{\text{sa},k+1} - T_{\text{ma},k})| + h_{fg} \cdot \max(0, \omega_{\text{ma},k} - {\omega}_{\text{sa},k+1}) \right]\\
&= {m}_{ma,k} \left[c_{pa}|\left({T}_{\text{sa},k+1} - \left(\gamma_{m,k} T_{o,k} + (1-\gamma_{m,k}) T^\star_{z,k}\right)\right)| + h_{fg} \cdot \max\left(0, \left(\gamma_{m,k}\omega_{o,k} + (1-\gamma_{m,k})\omega^\star_{z,k}\right) - {\omega}_{\text{sa},k+1}\right) \right]
\end{align}

Substituting the Kalman filter estimate $\mu_k$
\begin{align}
Q_{\text{coil,total},k+1} &= {m}_{ma,k} \left[c_{pa}|\left({T}_{\text{sa},k+1} - \left(\gamma_{m,k} T_{o,k} + (1-\gamma_{m,k}) \widehat{T}^\star_{z,k}\right)\right)| + h_{fg} \cdot \max\left(0, \left(\gamma_{m,k}\omega_{o,k} + (1-\gamma_{m,k})\widehat{\omega}^\star_{z,k}\right) - {\omega}_{\text{sa},k+1}\right) \right]
\end{align}
